In [5]:
import torch
import torch.nn as nn
from pytorch_msssim import ssim
from ignite.metrics import SSIM
import torch.optim as optim
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [6]:
from PIL import Image
from torchvision import transforms as T

image = Image.open('bus.jpg').convert('RGB')

from torchvision import transforms as T

transform = T.Compose([T.ToTensor(), T.Resize((224, 224))])
image = transform(image).to(device)[None,:,:,:]
image.mean()

/mnt/hdd3tb/Users/minhnqn/.conda/envs/py310/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(0.4595, device='cuda:0')

In [ ]:
class GcBlock(nn.Module):
    def __init__(self, c = 3):
        super(GcBlock, self).__init__()
        # Định nghĩa các lớp convolution và pooling
        self.Wk = nn.Conv2d(in_channels=c, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.softmax2d = nn.Softmax2d()
        self.Wv1 = nn.Conv2d(in_channels=c, out_channels=c//2, kernel_size=1, stride=1)
        self.layer_norm = nn.LayerNorm([c//2, 1, 1]) 
        self.relu = nn.ReLU(inplace=True)
        self.Wv2 = nn.Conv2d(in_channels=c//2, out_channels=c, kernel_size=1, stride=1)

    def forward(self, x):
        b, c, h, w = x.size()
        x1 = self.Wk(x) # b, 1, H, W
        x1 = x1.view(b, 1, h*w) # b, 1, w*h
        x1 = x1.unsqueeze(1).permute(0, 3, 1, 2) # b, h*w, 1, 1
        x1 = self.softmax2d(x1)
        x1 = x1.view(b, h*w, 1) # b, h*w, 1
        x_ = x.view(b, c, h*w) # b, c, h*w
        # print(x_.shape, x1.shape)
        y1 = torch.matmul(x_, x1, ).view(b, c, 1, 1) #b, c, 1, 1

        x2 = self.Wv1(y1) # b, c//2, 1, 1
        x2 = self.layer_norm(x2) # b, c//2, 1, 1 
        x2 = self.relu(x2) # b, c//2, 1, 1
        x2 = self.Wv2(x2) # b, c, 1, 1

        output = x + x2 # b, c, w, h
        return output

In [ ]:
model = GcBlock(c=3).to(device)
output = model(image)